In [1]:
import os

In [2]:
%pwd

'/Users/manraj/Documents/GitHub/text-summariser-E2E-project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/manraj/Documents/GitHub/text-summariser-E2E-project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        return DataIngestionConfig(
            root_dir=Path(self.config.data_ingestion.root_dir),
            source_URL=self.config.data_ingestion.source_URL,
            local_data_file=Path(self.config.data_ingestion.local_data_file),
            unzip_dir=Path(self.config.data_ingestion.unzip_dir)
        )

In [8]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(self.config.source_URL, self.config.local_data_file)
            logger.info(f"Downloaded data {filename} with following info: {headers}")
        else:
            logger.info(f"Data file {get_size(Path(self.config.local_data_file))} already exists")
            
    def extract_zip_file(self):
        """ Extracts the downloaded zip file to the specified unzip_dir """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Extracted zip file to {unzip_path}")

In [10]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config = data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-12-19 14:39:16,150]: INFO: common: Successfully read YAML file: config/config.yaml
[2024-12-19 14:39:16,152]: INFO: common: Successfully read YAML file: params.yaml
[2024-12-19 14:39:16,153]: INFO: common: Created directory: artifacts
[2024-12-19 14:39:16,154]: INFO: common: Created directory: artifacts/data_ingestion
[2024-12-19 14:39:18,151]: INFO: 266140058: Downloaded data artifacts/data_ingestion/data.zip with following info: Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 6647:E76FB:A94C61:BC79CA:67649AC4
Accept-Ranges: bytes
Date: Thu, 19 Dec 2024 22:39:16 GMT
Via: 1.1 varnish
X-Served-By: cache-yyc1430031-YYC
X-Cache: HIT
X